In [1]:
import pandas as pd
import json
import pyproj
from shapely.geometry import shape, Point
from shapely.ops import transform
from functools import partial
import time
start_time = time.time()

In [2]:
#Load DC Zillow Neighborhood GeoJSON file
with open('zillow_nb_dc.geojson', 'r') as jsonFile:
    dc_zil_nbhd = json.load(jsonFile)    

#Load DC business license data
dc_biz = pd.read_csv("Basic_Business_Licenses.csv", low_memory = False)
dc_biz['neighborhood'] = ''

dc_biz.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185688 entries, 0 to 185687
Data columns (total 45 columns):
BILLING_ADDRESS                   45488 non-null object
BILLING_ADDRESS_CITY_STATE_ZIP    185653 non-null object
AGENT_ENTITY                      95419 non-null object
AGENT_FIRST_NAME                  91061 non-null object
AGENT_LAST_NAME                   91090 non-null object
AGENT_MIDDLE_NAME                 15013 non-null object
AGENT_PHONE                       128663 non-null object
SITE_ADDRESS                      185021 non-null object
BBL_CITY_STATE_ZIP                185688 non-null object
BILLING_NAME                      66689 non-null object
CUSTOMER_NUMBER                   185688 non-null int64
DC_ADDR_FLAG                      185688 non-null object
LICENSECATEGORY                   185688 non-null object
ENTITY_NAME                       163208 non-null object
FIRST_NAME                        50274 non-null object
LAST_NAME                         50073 no

# 1.0 Basic Cleaning
### 1.1 License Types

In [3]:
## Public Health Food Establishments account for 9.7% of licenses
100*round(dc_biz['LICENSE_CATEGORY_TEXT'].value_counts()/len(dc_biz),3)

Housing: Residential            35.6
General Business                19.5
General Service and Repair      14.1
Public Health Food Establish     9.7
Motor Veh Sales Serv Repair      5.6
General Sales                    4.8
Environmental Materials          3.9
Inspected Sales and Services     1.6
Public Health Public Accomm      1.6
Entertainment                    1.5
Public Health Pharma             1.3
Housing: Transient               0.5
Employment Services              0.3
Public Health Funeral Est        0.1
Manufacturing                    0.0
Public Health Food Wholesale     0.0
Name: LICENSE_CATEGORY_TEXT, dtype: float64

In [4]:
## Restaurants and Delicatessesns are sub categories of Food Establishments 
## and respectively account for 4.1% and 2.6% of the dataset

100*round(
    dc_biz[dc_biz['LICENSE_CATEGORY_TEXT'] == 'Public Health Food Establish'] # filter data to food establish.
    ['LICENSECATEGORY'].value_counts() # count records by license category
          /len(dc_biz)                 # divide by #rows in df
    ,3)

Restaurant                  4.1
Delicatessen                2.6
Food Products               1.3
Grocery Store               0.8
Caterers                    0.4
Public School Cafeteria     0.3
Food Vending Machine        0.2
Bakery                      0.1
Marine Food Retail          0.0
Mobile Delicatessen         0.0
Ice Cream Manufacture       0.0
Commission Merchant Food    0.0
Candy Manufacturing         0.0
Name: LICENSECATEGORY, dtype: float64

In [5]:
dc_food = dc_biz[dc_biz['LICENSE_CATEGORY_TEXT'] == 'Public Health Food Establish'].copy()

len(dc_food)

18086

### 1.2 License Periods

In [6]:
## 98% of food establishments have licenses of 2 years or less, which makes me wonder if the license start dates are updated 
## each time that the business renews

# Add license end yr, start yr, and estimate of license length to df
dc_food['LICENSE_END_YR'] = dc_food['LICENSE_END_DATE'].str.slice(start = 0,stop = 4).fillna(0).astype(int)
dc_food['LICENSE_START_YR'] = dc_food['LICENSE_START_DATE'].str.slice(start = 0,stop = 4).fillna(0).astype(int)
dc_food['LICENSE_LENGTH'] = dc_food['LICENSE_END_YR'] - dc_food['LICENSE_START_YR']

dc_food['LICENSE_LENGTH'].value_counts().sort_index()/len(dc_food)

-2003    0.000055
 0       0.079785
 1       0.120093
 2       0.781710
 3       0.001272
 4       0.012551
 5       0.002488
 10      0.000276
 12      0.000055
 2003    0.001714
Name: LICENSE_LENGTH, dtype: float64

In [7]:
## I'll take a closer look at Washington DC's oldest restaurant
cols = ['SITE_ADDRESS','ENTITY_NAME'
        ,'LICENSE_ISSUE_DATE','LICENSE_START_DATE','LICENSE_END_DATE'
        ,'LICENSECATEGORY','LICENSESTATUS'
        , 'LATITUDE', 'LONGITUDE' 
       ]

## Old Ebbitt Grill has been in it's current location since 1983, 
## but it's license date is in 2018 so it appears that when licenses are renewed the license start date is updated
dc_food[dc_food['SITE_ADDRESS'].isin(['675 15TH ST NW '])][cols]

,SITE_ADDRESS,ENTITY_NAME,LICENSE_ISSUE_DATE,LICENSE_START_DATE,LICENSE_END_DATE,LICENSECATEGORY,LICENSESTATUS,LATITUDE,LONGITUDE
182624,675 15TH ST NW,OLD EBBITT GRILL,2018-02-28T00:00:00.000Z,2018-05-01T00:00:00.000Z,2020-04-30T00:00:00.000Z,Restaurant,Active,38.898056,-77.033329


In [8]:
## License end date is a bit messy, but seems reasonable since the early 2000s
## I'll take a closer look at YoY changes and the 7.1% of records that have a missing end date 
100*round(dc_food['LICENSE_END_YR'].value_counts().sort_index()/len(dc_food),3)

0        7.1
1998     0.0
1999     0.0
2000     1.0
2001     1.1
2002     2.1
2003     4.1
2004     3.3
2005     3.5
2006     4.2
2007     2.8
2008     3.7
2009     3.0
2010     3.7
2011     3.0
2012     3.4
2013     2.8
2014     3.2
2015     3.4
2016     4.5
2017     3.3
2018     3.9
2019     8.2
2020    15.1
2021     8.4
2022     1.3
2023     0.1
2025     0.0
2027     0.0
Name: LICENSE_END_YR, dtype: float64

In [9]:
### The YoY change in food establishments license end date is within +/- 33% throughout the past 15 years
dc_food_endyr = dc_food[(dc_food.LICENSE_END_YR >= 2004) & (dc_food.LICENSE_END_YR < 2019)]['LICENSE_END_YR'].value_counts().sort_index()

dc_food_endyr_YoY = 100*(round(dc_food_endyr/dc_food_endyr.shift(1),3)-1)
dc_food_endyr_YoY = dc_food_endyr_YoY[pd.notna(dc_food_endyr_YoY)]
print(dc_food_endyr_YoY)
print("Min:", min(dc_food_endyr_YoY))
print("Max:", max(dc_food_endyr_YoY))

2005     6.1
2006    20.3
2007   -33.7
2008    33.2
2009   -18.9
2010    24.1
2011   -18.7
2012    13.8
2013   -19.4
2014    14.4
2015     7.3
2016    31.9
2017   -26.2
2018    18.2
Name: LICENSE_END_YR, dtype: float64
Min: -33.699999999999996
Max: 33.20000000000001


In [10]:
### looks like the missing records are due to a bunch of records that were issued
### 10/04/2010 and 10/05/2010 and cancelled/expired
### Since the YoY changes seem reasonable, I'll assume these are errors and remove them from the dataset
dc_food[dc_food['LICENSE_END_YR'] == 0].groupby(['LICENSE_ISSUE_DATE','LICENSESTATUS'])['ENTITY_NAME'].count()

LICENSE_ISSUE_DATE        LICENSESTATUS
2010-10-04T00:00:00.000Z  Cancelled        288
                          Expired          623
2010-10-05T00:00:00.000Z  Cancelled        151
                          Expired          212
                          Transferred        3
2010-10-14T00:00:00.000Z  Expired            1
2013-11-29T00:00:00.000Z  Cancelled          0
2014-09-10T00:00:00.000Z  Cancelled          1
Name: ENTITY_NAME, dtype: int64

In [11]:
dc_food = dc_food[dc_food['LICENSE_END_YR'] >= 2005]
len(dc_food)

14710

# 2.0 Validations
### 2.1 Benchmark
I'll compare the number of active restaurants licences in 2019 and benchmark against Destination DC's count of 2,233 restaurants
https://washington.org/dc-information/washington-dc-facts#Restaurant%20Facts

In [12]:
##I'll consider "Active" and "Ready to Renew" as active restaurants
dc_food[(dc_food.LICENSECATEGORY == 'Restaurant')]['LICENSESTATUS'].value_counts()

Expired                 2690
Active                  2213
Cancelled                998
Transferred              169
Ready to Renew           115
Deficient                 25
Lapsed                    25
Ready to Batch Print      11
Revoked                    7
Pending                    3
Closed                     2
Name: LICENSESTATUS, dtype: int64

In [13]:
## I'm within 2% of Destination DC's restaurant count
cnt = len(dc_food[(dc_food.LICENSECATEGORY == 'Restaurant') 
           & (dc_food.LICENSE_END_DATE >= '2019-09-01') 
           & (dc_food.LICENSE_START_DATE <= '2019-09-01') 
           & (dc_food.LICENSESTATUS.isin(['Active','Ready to Renew']))       
          ])

print(cnt)

print(cnt/2233 - 1)

2237
0.0017913121361397


### 2.2 Spot Checks

In [14]:
## Filtering dataset to a specific section of K Street NW 
## to see how L'Hommage Bistro which closed in 2017 
## and La Betty (which opened in 2019) appear
K_St_1 = dc_food[(dc_food.LATITUDE.apply(lambda x: round(x, 3)) <= 38.903) & 
                  (dc_food.LATITUDE.apply(lambda x: round(x, 3)) >= 38.902) & 
                  (dc_food.LONGITUDE.apply(lambda x: round(x, 3)) >= -77.019) & 
                  (dc_food.LONGITUDE.apply(lambda x: round(x, 3)) <= -77.015)]

## there's a ton of records for 490 L St Nw w/ same entity name, which is making it hard to review
K_St_1[K_St_1['LICENSE_END_YR'] >= 2016][cols].sort_values(by=['SITE_ADDRESS'], ascending = False).head(40)

,SITE_ADDRESS,ENTITY_NAME,LICENSE_ISSUE_DATE,LICENSE_START_DATE,LICENSE_END_DATE,LICENSECATEGORY,LICENSESTATUS,LATITUDE,LONGITUDE
41944,919 5TH ST NW,COMMUNAL RESTAURANT LLC.,2018-05-03T00:00:00.000Z,2018-05-01T00:00:00.000Z,2020-04-30T00:00:00.000Z,Restaurant,Active,38.901924,-77.018643
47683,919 5TH ST NW,SILO RESTAURANT,2017-11-27T00:00:00.000Z,2017-12-01T00:00:00.000Z,2019-11-30T00:00:00.000Z,Restaurant,Active,38.901924,-77.018643
43624,901 4TH ST NW,LETTA DC LLC.,2019-04-12T00:00:00.000Z,2019-04-01T00:00:00.000Z,2021-03-31T00:00:00.000Z,Restaurant,Active,38.901685,-77.015895
133228,500 K ST NW,J.S. & W.C INCORPORATED,2018-04-03T00:00:00.000Z,2016-03-01T00:00:00.000Z,2018-02-28T00:00:00.000Z,Food Products,Expired,38.902271,-77.019169
10401,490 L ST NW,NAI Saturn Eastern LLC,2019-04-05T00:00:00.000Z,2019-06-01T00:00:00.000Z,2021-05-31T00:00:00.000Z,Grocery Store,Active,38.903489,-77.018353
47901,490 L ST NW,NAI Saturn Eastern LLC,2019-04-05T00:00:00.000Z,2019-06-01T00:00:00.000Z,2021-05-31T00:00:00.000Z,Grocery Store,Active,38.903489,-77.018353
163671,490 L ST NW,SAFEWAY #2737,2015-05-08T00:00:00.000Z,2014-12-01T00:00:00.000Z,2016-11-30T00:00:00.000Z,Restaurant,Cancelled,38.903489,-77.018353
153836,490 L ST NW,NEW ALBERTSON'S INC,2019-04-05T00:00:00.000Z,2017-05-01T00:00:00.000Z,2019-04-30T00:00:00.000Z,Restaurant,Cancelled,38.903489,-77.018353
152121,490 L ST NW,NEW ALBERTSON'S INC,2019-04-05T00:00:00.000Z,2017-05-01T00:00:00.000Z,2019-04-30T00:00:00.000Z,Restaurant,Cancelled,38.903489,-77.018353
64533,490 L ST NW,SAFEWAY #2737,2015-05-08T00:00:00.000Z,2014-12-01T00:00:00.000Z,2016-11-30T00:00:00.000Z,Restaurant,Cancelled,38.903489,-77.018353


In [15]:
## Per review of data in Excel
## it seems that license_category_code, objectid, bbl_license_fact_id, 
## misspellings in agent entity name and differences in specificity of latitude, longitude, can cause duplicate cancelled records
dc_food[dc_food['ENTITY_NAME'].isin(['SAFEWAY #2737',"NEW ALBERTSON'S INC"])].to_csv("dups.csv", index = False)

In [16]:
## I'm going to limit the dataset to just the columns that I am interesetd in so that the duplicate 
## cancelled records do not skew my analysis
cols = ['SITE_ADDRESS','ENTITY_NAME'
        ,'LICENSE_ISSUE_DATE','LICENSE_START_DATE','LICENSE_END_DATE'
        ,'LICENSECATEGORY','LICENSESTATUS'
        , 'LATITUDE', 'LONGITUDE'
        , 'WARD','ANC','SMD','DISTRICT','PSA','NEIGHBORHOODCLUSTER',
        'BUSINESSIMPROVEMENTDISTRICT','MAINSTREET',
        'neighborhood','LICENSE_END_YR','LICENSE_START_YR','LICENSE_LENGTH']

dc_food = dc_food[cols]

In [17]:
## rounding lat and long to 6 decimals
dc_food['LATITUDE'] = dc_food['LATITUDE'].round(decimals = 6)
dc_food['LONGITUDE'] = dc_food['LONGITUDE'].round(decimals = 6)

In [18]:
dc_food_final = dc_food.drop_duplicates()
print(len(dc_food))
print(len(dc_food_final))

14710
13106


In [19]:
## Filtering dataset to a specific section of K Street NW 
## to see how L'Hommage Bistro which closed in 2017 
## and La Betty (which opened in 2019) appear
K_St = dc_food_final[(dc_food_final.LATITUDE.apply(lambda x: round(x, 4)) <= 38.9028) & 
                  (dc_food_final.LATITUDE.apply(lambda x: round(x, 4)) >= 38.9020) & 
                  (dc_food_final.LONGITUDE.apply(lambda x: round(x, 3)) >= -77.019) & 
                  (dc_food_final.LONGITUDE.apply(lambda x: round(x, 3)) <= -77.017)]

## L'Hommage Bistro license ends 7/2017 and La Betty starts 1/2019
K_St[K_St['LICENSE_END_YR'] >= 2016][cols].sort_values(by=['SITE_ADDRESS'], ascending = True).head(100)

,SITE_ADDRESS,ENTITY_NAME,LICENSE_ISSUE_DATE,LICENSE_START_DATE,LICENSE_END_DATE,LICENSECATEGORY,LICENSESTATUS,LATITUDE,LONGITUDE,WARD,...,SMD,DISTRICT,PSA,NEIGHBORHOODCLUSTER,BUSINESSIMPROVEMENTDISTRICT,MAINSTREET,neighborhood,LICENSE_END_YR,LICENSE_START_YR,LICENSE_LENGTH
134099,1025 5TH ST NW,BUSBOYS AND POETS- 5TH & K,2019-05-01T00:00:00.000Z,2017-04-01T00:00:00.000Z,2019-03-31T00:00:00.000Z,Restaurant,Expired,38.902815,-77.018704,6.0,...,6E05,FIRST,101.0,Cluster 8,Mount Vernon Triangle CID,NaN,,2019,2017,2
46814,420 K ST NW,LA BETTY,2019-01-03T00:00:00.000Z,2019-01-01T00:00:00.000Z,2020-12-31T00:00:00.000Z,Restaurant,Active,38.902210,-77.016901,6.0,...,6E05,FIRST,101.0,Cluster 8,Mount Vernon Triangle CID,NaN,,2020,2019,1
149733,440 K ST NW,OUTPOSTDC LLC,2019-05-14T00:00:00.000Z,2019-05-01T00:00:00.000Z,2021-04-30T00:00:00.000Z,Restaurant,Active,38.902281,-77.017458,6.0,...,6E05,FIRST,101.0,Cluster 8,Mount Vernon Triangle CID,NaN,,2021,2019,2
161901,449 K ST NW,RAY'S THE STEAKS,2019-06-03T00:00:00.000Z,2017-05-01T00:00:00.000Z,2019-04-30T00:00:00.000Z,Delicatessen,Expired,38.902832,-77.017776,6.0,...,6E05,FIRST,101.0,Cluster 8,Mount Vernon Triangle CID,NaN,,2019,2017,2
39054,450 K ST NW,BUSBOYS & POETS,2018-08-28T00:00:00.000Z,2018-08-01T00:00:00.000Z,2020-07-31T00:00:00.000Z,Restaurant,Active,38.902275,-77.017966,6.0,...,6E05,FIRST,101.0,Cluster 8,Mount Vernon Triangle CID,NaN,,2020,2018,2
129330,450 K ST NW,L'HOMMAGE BISTRO FRANCAIS,2017-09-01T00:00:00.000Z,2015-08-01T00:00:00.000Z,2017-07-31T00:00:00.000Z,Restaurant,Expired,38.902275,-77.017966,6.0,...,6E05,FIRST,101.0,Cluster 8,Mount Vernon Triangle CID,NaN,,2017,2015,2
183355,453 K ST NW,MANDU LLC.,2019-01-07T00:00:00.000Z,2019-01-01T00:00:00.000Z,2020-12-31T00:00:00.000Z,Restaurant,Active,38.902826,-77.017893,6.0,...,6E05,FIRST,101.0,Cluster 8,Mount Vernon Triangle CID,NaN,,2020,2019,1
134429,465 K ST NW,KUSHI IZAKAYA,2016-03-01T00:00:00.000Z,2014-02-01T00:00:00.000Z,2016-01-31T00:00:00.000Z,Restaurant,Expired,38.902823,-77.018192,6.0,...,6E05,FIRST,101.0,Cluster 8,Mount Vernon Triangle CID,NaN,,2016,2014,2
151398,465 K ST NW,ALTA STRADA-CITY VISTA LLC,2018-04-17T00:00:00.000Z,2018-03-01T00:00:00.000Z,2020-02-28T00:00:00.000Z,Restaurant,Active,38.902823,-77.018192,6.0,...,6E05,FIRST,101.0,Cluster 8,Mount Vernon Triangle CID,NaN,,2020,2018,2
12366,476 K ST NW,476 K LLC,2018-05-10T00:00:00.000Z,2018-05-01T00:00:00.000Z,2020-04-30T00:00:00.000Z,Restaurant,Active,38.902266,-77.018702,6.0,...,6E05,FIRST,101.0,Cluster 8,Mount Vernon Triangle CID,NaN,,2020,2018,2


In [20]:
dc_food_final = dc_food_final[dc_food_final['LONGITUDE'].notna()]
len(dc_food_final)

12327

# 3.0 Transformations
### 3.1 Encode Neighborhood

In [21]:
long = dc_food_final.columns.get_loc('LONGITUDE')
lat = dc_food_final.columns.get_loc('LATITUDE')

## use shapely to check if lat/lon is within the zillow neighborhood shape
for i in range(len(dc_food_final)):
    point = Point(dc_food_final.iloc[i,long],dc_food_final.iloc[i,lat]) ## Longitude, Latitude

    for feature in dc_zil_nbhd['features']:
        polygon = shape(feature['geometry'])
        if polygon.contains(point):
            dc_food_final.iloc[i, dc_food_final.columns.get_loc('neighborhood')] = feature['properties']['name']
            
dc_food_final.to_csv("DC_FoodEstablish_Since2005.csv", index = False) ## write the data so we don't have to re-run this every time
        
dc_food_final.head()

,SITE_ADDRESS,ENTITY_NAME,LICENSE_ISSUE_DATE,LICENSE_START_DATE,LICENSE_END_DATE,LICENSECATEGORY,LICENSESTATUS,LATITUDE,LONGITUDE,WARD,...,SMD,DISTRICT,PSA,NEIGHBORHOODCLUSTER,BUSINESSIMPROVEMENTDISTRICT,MAINSTREET,neighborhood,LICENSE_END_YR,LICENSE_START_YR,LICENSE_LENGTH
696,500 PENN ST NE,ANGELIKA FILM CENTER UNION MARKET LLC,2018-07-27T00:00:00.000Z,2018-06-01T00:00:00.000Z,2020-05-31T00:00:00.000Z,Delicatessen,Active,38.910537,-76.997168,5.0,...,5D01,FIFTH,501.0,Cluster 23,NaN,NaN,Gallaudet,2020,2018,2
2104,3427 CONNECTICUT AVE NW,STREETS MARKET,2018-05-22T00:00:00.000Z,2018-05-01T00:00:00.000Z,2020-04-30T00:00:00.000Z,Delicatessen,Active,38.935235,-77.057885,3.0,...,3C04,SECOND,203.0,Cluster 15,NaN,NaN,Cleveland Park,2020,2018,2
2105,1223 GOOD HOPE RD SE,GOOH LLC.,2019-05-17T00:00:00.000Z,2019-05-01T00:00:00.000Z,2021-04-30T00:00:00.000Z,Delicatessen,Active,38.867024,-76.987905,8.0,...,8A06,SEVENTH,701.0,Cluster 28,Anacostia BID,NaN,Anacostia,2021,2019,2
2106,503 KENNEDY ST NW,A AND A CONVENIENCE STORE INC.,2018-02-21T00:00:00.000Z,2018-02-01T00:00:00.000Z,2020-01-31T00:00:00.000Z,Food Products,Active,38.956641,-77.020141,4.0,...,4D01,FOURTH,403.0,Cluster 18,NaN,Uptown,Brightwood Park,2020,2018,2
2107,1613 MONTELLO AVE NE,COMPACT SUPERMARKET,2019-03-04T00:00:00.000Z,2019-03-01T00:00:00.000Z,2021-02-28T00:00:00.000Z,Grocery Store,Active,38.907359,-76.986621,5.0,...,5D02,FIFTH,506.0,Cluster 23,NaN,NaN,Trinidad,2021,2019,2


### 3.2 Calculate Area

In [22]:
neighb_areas = dict()

for feature in dc_zil_nbhd['features']:
    geom = feature['geometry']
    name = feature['properties']['name']
    
    s = shape(geom)
    proj = partial(pyproj.transform, pyproj.Proj(init='epsg:4326'),
               pyproj.Proj(init='epsg:3857'))

    s_new = transform(proj, s)

    projected_area = transform(proj, s).area
    neighb_areas[name] = projected_area / 2589988.1103

In [23]:
neighb_area = pd.DataFrame.from_dict(neighb_areas, orient = 'index')

In [24]:
neighb_area.reset_index(level=0, inplace=True)
neighb_area.rename(columns={"index": "name", 0: "sq_mi"}, inplace = True)
neighb_area.to_csv("neigbh_area.csv", index = False)

In [25]:
(time.time() - start_time)/60

1.119023068745931